In [6]:
!pip install -U -r requirements.txt


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.yandex import YandexGPTEmbeddings

loader = Docx2txtLoader("output.docx")

data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=100, 
    add_start_index=True, 
    strip_whitespace=True, 
)

docs_processed = text_splitter.split_documents(data)


folder_id = 'b1g72uajlds114mlufqi'
api_key = 'AQVN0RWGrzTjNpeuo3e9XkJ1KC0PlUJRmm2afUV4'
embeddings = YandexGPTEmbeddings(
        api_key=api_key, folder_id=folder_id)

In [ ]:


KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embeddings, distance_strategy=DistanceStrategy.COSINE
)

In [ ]:
KNOWLEDGE_VECTOR_DATABASE.save_local("faiss_index")

In [2]:
KNOWLEDGE_VECTOR_DATABASE = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [3]:
from langchain_community.llms import YandexGPT
from langchain.chains import RetrievalQA
from langchain_core.prompts.prompt import PromptTemplate

rag_k = 5

template = """Вы - ДокуБот, помощник пользователей. Вы дружелюбны, умны и трудолюбивы. Ваша основная задача - помогать пользователям находить ответы на их вопросы, используя документацию RuStore. Вот что вам нужно делать:

Помощь в работе с документацией

Помогайте пользователям находить и понимать информацию в документации RuStore. Для этого:
Разъясняйте непонятные моменты.
Указывайте на разделы документации, содержащие нужную информацию.
Объясняйте, как правильно использовать предоставленные данные.
Поддержка программирования

Если вопрос связан с программированием, вы можете:
Дополнять существующий код.
Исправлять ошибки в коде.
Писать новый код на основе заданного контекста и вопроса.

Для этого:
Анализируйте предоставленный код и выявляйте ошибки.
Предлагайте оптимизации и улучшения.
Пишите код, соответствующий современным стандартам и лучшим практикам.
Уведомления об обновлениях

Если пользователи используют устаревшие версии API или кода, сообщайте им об этом и предоставляйте исправленную, актуальную версию. Для этого:
Мониторьте текущие версии API и инструментария.
Обновляйте пользователей о последних изменениях и новых функциях.
Предоставляйте примеры кода, соответствующие последним версиям.
Форматирование в формате Markdown

Представляйте свои ответы в формате Markdown. Если ваш ответ содержит код, заключайте его в три тильды (~~~) с обеих сторон для правильного форматирования. Для этого:
Оборачивайте блоки кода в три тильды.
Используйте заголовки и списки для структурирования информации.
Поддерживайте читаемость и понятность текста.

Если ты понимаешь что не сможешь внятно ответить на вопрос, или у тебя недостаточно знаний - попробуй сначала ответить на вопрос, а если вообще не можешь -ответь ТОЛЬКО следующей фразой: “Ой! Кажется мне нужна помощь моих коллег разработчиков для ответа на такой вопрос”.

При формулировании ответа учитывайте приведенные контексты:

{context}

Вопрос:

{question}
"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
llm = YandexGPT(api_key=api_key, folder_id=folder_id)

qa = RetrievalQA.from_chain_type(
        llm,
        retriever=KNOWLEDGE_VECTOR_DATABASE.as_retriever(search_kwargs={'k': rag_k}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

In [4]:
user_input = 'Что можно написать для RuStore на языке Flutter? Приведи пример кода для чего-то'
output = qa(
    {"query": user_input})

/Users/valery/hack_cp_june/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [5]:
print(f"q: {output['query']} \n\n a: {output['result']}")

q: Что можно написать для RuStore на языке Flutter? Приведи пример кода для чего-то 

 a: Могу предложить следующий пример кода на Flutter, который демонстрирует интеграцию с RuStore для показа процесса обновления приложения:

```dart
import 'package:flutter/material.dart';
import 'dart:async';

import 'package:rustore_updates_sdk/rustore_updates_service.dart'; // Импортируем пакет для интеграции с SDK
import 'rustore_update_service_client.dart';  // Импортируем класс для работы
                                               // с сервисным клиентом
import 'update_service.dart';     // Импортируем модель для управления обновлением

void main() async {
  WidgetsFlutterBinding.ensureInitialized();

  // Инициализируем сервисный клиент
  final serviceClient = RustoreUpdateServiceClient(
    'https://update-service.rustoreapp.com',
    secretKey: 'secret_key',
  );

  runApp(MyApp());
}

class MyApp extends StatefulWidget {
  @override
  _MyAppState createState() => _MyAppState();
}

// Это